In [33]:
import zipfile
from tqdm import tqdm
import cv2
from glob import glob
import os,shutil
import numpy as np
from pydicom import dcmread

In [15]:
try:
    os.rename('download','download.zip')
except:
    None

file=zipfile.ZipFile('download.zip')
        # 解压文件
        # print('开始解压...')
file.extractall()
        # print('解压结束。')
        # 关闭文件流
file.close()
# 压缩文件路径

for i in tqdm(range(1,21)):
    
    for name in ['MASKS','PATIENT']:
        zip_path='3Dircadb1/3Dircadb1.{}/{}_DICOM.zip'.format(i,name)
        # 文件存储路径
        save_path = '3Dircadb1/3Dircadb1.{}/'.format(i)

        # 读取压缩文件
        file=zipfile.ZipFile(zip_path)
        # 解压文件
        #print('开始解压...')
        file.extractall(save_path)
        #print('解压结束。')
        # 关闭文件流
        file.close()
        


100%|██████████| 20/20 [01:19<00:00,  3.96s/it]


In [38]:
if os.path.exists('CTDircadb1/mask/') == False and os.path.exists('CTDircadb1/mask/') == False:
    os.mkdir('CTDircadb1/mask/')
    os.mkdir('CTDircadb1/origin/')

In [39]:
from pydicom import dcmread
import SimpleITK as sitk

# os.rename()
for i in tqdm(range(1,21)):
    
    for name in ['MASKS','PATIENT']:
        if name == 'MASKS':
            doc_list = glob('3Dircadb1/3Dircadb1.{}/{}_DICOM/bone/*'.format(i,name))
            out_path = 'CTDircadb1/mask/'

            for mask_path in doc_list:

                img_name = mask_path.rsplit('/',1)[1]
                shutil.copyfile(mask_path,out_path+str(i)+'_'+img_name+'.dcm')

        else:
            doc_list = glob('3Dircadb1/3Dircadb1.{}/{}_DICOM/*'.format(i,name))
            out_path = 'CTDircadb1/origin/'
            
            for origin_path in doc_list:
                
                img_name = origin_path.rsplit('/',1)[1]
                shutil.copyfile(origin_path,out_path+str(i)+'_'+img_name+'.dcm')
                

100%|██████████| 20/20 [00:18<00:00,  1.07it/s]


In [40]:
for name in ['mask','origin']:
    
        path_res = 'CTDircadb1/{}/'.format(name)
        file_list = glob(path_res+'/*.dcm')
        for i in tqdm(file_list):
            a = dcmread(i)
            img_arr = a.pixel_array
            if name == 'mask':
                img_arr = np.where(img_arr < 1, img_arr, 255)

            cv2.imwrite(i[:-3]+'png',img_arr)


100%|██████████| 2823/2823 [00:13<00:00, 213.99it/s]


In [41]:
for name in ['mask','origin']:
    path_res = 'CTDircadb1/{}/'.format(name)    
    for j in glob(path_res+'/*.dcm'):
        os.remove(j)

In [29]:
import os
from glob import glob
from random import sample
import shutil

path_test_ori = 'CTDircadb1/test/origin/'
path_test_mask = 'CTDircadb1/test/mask/'

if os.path.exists('CTDircadb1/test/') == False :
    os.mkdir('CTDircadb1/test/')
    os.mkdir(path_test_ori)
    os.mkdir(path_test_mask)


In [30]:
radio = 0.1

ori_path = glob(  path_test_ori.replace('/test','')  +'/*.png')
mask_path = glob(path_test_mask.replace('/test','')  +'/*.png')

ori_path.sort()
mask_path.sort()

pic_name = [i.rsplit('/',1)[1] for i in ori_path]
pic_name.sort()

In [31]:
for name in sample(pic_name,int(len(pic_name)*radio)):
    for file in ['mask','origin']:
        shutil.move('CTDircadb1/{}/'.format(file) + name,'CTDircadb1/test/{}/'.format(file) + name)
        